# Jike-crawl

## 配置运行环境

请先参照 `README.md` 准备好运行环境。

- Python 3
- json
- requests

也可以直接运行下面代码块。

In [ ]:
! pip install -r ../requirements.txt

## 准备工作

1. 进入开发者模式。在 [即刻网页版](https://web.okjike.com/) 中登录自己的即刻账号，并进入个人主页。按 `F12` 打开开发者工具。切换到 `网络(Network)`，过滤 `Fetch/XHR`，刷新页面，此时下方会出现请求列表，罗列了请求的名称、状态等信息。

2. 获取cookie。在请求列表中，任选一个名称为 `graphql` 的请求，单击 `标头(Headers)`，找到 `cookie` 字段并复制全文，粘贴到 `config/cookies.txt`。

    ![devtools](../img/devtools_cn.png)

3. 获取id。在请求列表中，任选一个名称为 `profile?username=...` 的请求，单击 `响应(Response)`，复制 `username` 字段:后的内容，粘贴到下面代码块的 `user_id` 并运行代码块。

In [ ]:
# -*- coding:utf8 -*-

from crawl import *
from data_analysis import *
from delete_posts import *
from like_posts import *

# Replace with your own Jike user id.
user_id = "D5560B5D-7448-4E1A-B43A-EC2D2C9AB7EC"

#CURR_TIME = datetime.now(GMT8())  # current time
done("user_id updated as", user_id, "at", CURR_TIME)


## 抓取动态和消息

### 配置

下面的代码块罗列了可供配置的变量。**修改之后切记重新运行此代码块！**

- `noti_path` 是存放抓取的消息的文件，默认在 `data/notifications.json`；
- `post_path` 是存放抓取的动态的文件，默认在 `data/posts.json`；
- `b_save_pics` 设置是否保存动态中的图片，默认不保存；
- 您可以通过修改 `start_time` 和 `end_time` 来保存特定时间段内发布的动态，默认保存全部；
  您可以像下面所示构建 `datetime` 对象：

    ```python
    # class datetime.datetime(year, month, day, hour=0, minute=0, second=0, microsecond=0, tzinfo=None, *, fold=0)
    datetime(2021, 1, 1, tzinfo=GMT8()) #  2021/01/01 00:00:00.000 (+08:00)
    datetime(2021, 1, 1, 12, 13, 14, 15, tzinfo=GMT8()) # 2021/01/01 12:13:14.000015 (+08:00)
    ```

    或者使用 `CURR_TIME` 或/和 `BASE_TIME` 加上/减去 `timedelta`，下面展示了拉取 **30 天之前**的动态的设置。

    ```python
    # class datetime.timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
    time_delta = timedelta(days=30)
    end_time = CURR_TIME - time_delta
    ```

- `noti_record_limit` 和 `post_record_limit` 分别是抓取的消息和动态的数量限制；
  下面展示了爬取 2022/01/01 至 2022/06/26 的 **10 个最近的**动态的配置

    ```python
        post_start_time = datetime(2022, 1, 1, tzinfo=GMT8())
        post_end_time = datetime(2022, 6, 26, tzinfo=GMT8())
        post_record_limit = 10
        crawl_posts(user_id, post_path, "a", post_record_limit, post_start_time, post_end_time)
    ```

In [ ]:
noti_path = os.path.join(DIR_PATH, "data/notifications.json")
noti_start_time = BASE_TIME  # datetime
noti_end_time = CURR_TIME   # datetime
noti_record_limit = None    # int or None

post_path = os.path.join(DIR_PATH, "data/posts.json")
post_start_time = BASE_TIME  # datetime
post_end_time = CURR_TIME   # datetime
post_record_limit = None    # int or None

b_save_pics = False  # Bool
update = True  # Bool


# RERUN THIS CODE BLOCK TO UPDATE THE CONFIGURATION
#CURR_TIME = datetime.now(GMT8())  # current time
done("configurations updated at", CURR_TIME)

### 抓取消息

In [ ]:
crawl_notifications(noti_path, "a", b_save_pics, noti_record_limit, noti_start_time, noti_end_time, update)

### 抓取动态

#### 重排
若先前动态中存在置顶未正确插入，需要运行下面代码块重新排列。之后再抓取动态将会从以存储的最近动态开始。

In [ ]:
sort_nodes(post_path)

#### 抓取动态

如需重新抓取请指定空文件。

In [ ]:
crawl_posts(user_id, post_path, "a", b_save_pics, post_record_limit, post_start_time, post_end_time, update)

## 数据分析

必须先保存数据，方可进行数据分析。

运行下面代码块分析获得的数据。

### 分析动态

In [ ]:
summarize_posts(post_path)

### 分析消息

In [ ]:
summarize_notifications(noti_path)

## 删除动态

必须先保存动态数据，方可删除动态。

此操作仅可根据本地保存的数据进行删除。即在新发动态后，如果没有重新拉取，则无法删除。

1. 若想删除所有动态，请跳过此步。若想修改动态删除的时间范围，请将下面代码块中的 `start_time`（开始时间）和 `end_time`（结束时间）修改为所需的日期(也可详细设定时间）。`limit` 是数量限制，注意动态从后往前删，最新发布的先删。先不运行代码块。

2. 确认上面配置中的 `post_path` 是储存在本地的 `posts.json` 的路径并且该代码块已经运行。

3. 打开 `src/delete_posts.py` 取消 `clear()` 函数中 **DANGER ZONE** 中的这行注释（按 `Ctrl` + `/` 或删掉这行前面的 `# `）。

    ```python
    ################# DANGER ZONE ##################
    ################################################
    # uncomment next line to remove all your posts #
    # remove(id) # remove posts by id              #
    ################################################
    ```

4. 运行下面代码块。
5. 若出现
   ```shell
   Connection error (MaxRetryError("HTTPSConnectionPool(...): Max retries exceeded with url: /api/graphql (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at ...>, 'Connection to ... timed out. (connect timeout=None)'))"))
   ```
   重新运行即可。

In [ ]:
# operate posts created during 2021/12/01 and 2021/12/31
# start_time = datetime(2021, 1, 1, tzinfo=GMT8())
# end_time = datetime(2021, 12, 31, tzinfo=GMT8())

# operate posts created before 30 days ago
# class datetime.timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
# time_delta = timedelta(days=30)
# end_time = CURR_TIME - time_delta

# operate all posts 
start_time = BASE_TIME  # datetime
end_time = CURR_TIME    # datetime
limit = None            # int or None

clear(post_path, start_time, end_time, limit)

## 自动点赞

确定自动点赞对象的方法有两种，一种是在 `config/user_name_list` 中输入用户的昵称，然后运行 `update_user_id_list()`，亦即以下代码块第 8 行，此函数会自动搜寻对应的真实用户名 `username`，然后写入 `config/user_id_list.txt`。

另一种是抓取关注列表然后手动选择。详见 [用户关系](#用户关系) 一章。

总之将对象用户 `username` 列表放进 `config/user_id_list.txt` 后就可以开始操作，下面代码块中提供了一些使用方法，`like` 和 `unlike` 相似，以 `like` 为例，他有两种调用方式。

一种逻辑是进入该用户主页然后从上到下（包括置顶）点赞，这样的调用方式如第 10 行所示，后面三个有默认参数，可以不用写。

另一种逻辑是在你的动态页面接收你还没看过的动态，然后筛选出在用户列表中的发表者的文章点赞，`miss_feed_only` 必须为 `True`，如第 13 行所示。

In [ ]:
start_time = CURR_TIME - timedelta(days=1)  # datetime
end_time = CURR_TIME    # datetime
record_limit = 5        # int or None
miss_feed_only = True

# user_id_list = read_list_file(os.path.join(
#     DIR_PATH, "config/user_id_list.txt"))
user_id_list = update_user_id_list()

like(user_id_list, record_limit=None, start_time=start_time, end_time=end_time) # only operate posted in 30 days
# unlike(user_id_list)

# like(user_id_list, miss_feed_only, record_limit, start_time, end_time)


## 用户关系

### 抓取关注列表和被关注列表

结果保存至 `data/relation`

In [ ]:
crawl_following(user_id)
crawl_follower(user_id)

### 交集和差集

结果保存至 `data/relation`

In [ ]:
get_mutual_following_list()

### 读取关注列表到 `config/`

在获取关注列表之后执行下面的代码块，将会读取关注列表到 `config/user_list_temp.txt`，在那可以删除不需要的用户。

In [ ]:
following_list = read_multi_json_file(os.path.join(DIR_PATH, "data/relation/following.json"))
save_list([user["screenName"] + "       " + user["username"]
                for user in following_list], os.path.join(DIR_PATH, "config/user_list_temp.txt"))

之后执行下面代码块将选中的用户列表读取到 `config/user_id_list.txt`，之后便可执行诸如 [自动点赞](#自动点赞) 等操作。

In [ ]:
load_following_list()

##  其他

### 清除 log

In [ ]:
clear_log()

### 重新装载 cookies

In [ ]:
refresh_cookies()

### 更新 `CURR_TIME`

In [ ]:
#CURR_TIME = datetime.now(GMT8())